# <b> Cross-validation & Search </b>

## <b> Problem Set: </b>


## <b> Soultion key: </b>


In [2]:
# labrary
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

#### <b> Data prepare </b>

In [3]:
wine =  pd.read_csv('https://bit.ly/wine-date')

wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


#### <b> Data preprocessing </b>
before sampling -> train data(80%), test date(20%)  
now sampling -> train data(60%), validation data(20%), test data(20%)   

In [4]:
# Check columns
col = list(wine.columns)
print("columns:", col, end="\n\n")

# divide input, target
wine_input =  wine[col[:3]].to_numpy()
wine_target = wine[col[3:][0]].to_numpy()

# data sampling data -> train(80), test(20)
sli = train_test_split(wine_input, wine_target, test_size= 0.2, random_state=42)
train_input = sli[0]
train_target = sli[2]
test_input = sli[1]
test_target = sli[3]

# data sampling train -> train_sub(60), validation(20)
sli = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
train_sub_input = sli[0]
traub_sub_target = sli[2]
valid_input = sli[1]  # validation
valid_target = sli[3]

print("     Train | Valid | Test  |")
print("Rate:", np.round(train_sub_input.shape[0]/wine_input.shape[0]*100), "| ", 
               np.round(valid_input.shape[0]/wine_input.shape[0]*100), "| ",
               np.round(test_input.shape[0]/wine_input.shape[0]*100), "| ", end="\n\n")

# Evaluation
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_sub_input, traub_sub_target)

print("Evaluation:")
print("Sub: ", dt.score(train_sub_input, traub_sub_target))
print("Test: ", dt.score(valid_input, valid_target))

# Overfitting in tran data set.

columns: ['alcohol', 'sugar', 'pH', 'class']

     Train | Valid | Test  |
Rate: 64.0 |  16.0 |  20.0 | 

Evaluation:
Sub:  0.9971133028626413
Test:  0.864423076923077


#### <b> Cross Validation </b>
k-fold cross validation

<b> example: 3-fold cross valiation </b>    
1step: [train set]  | [train set]  | [validation]  
2step: [train set]  | [validation] | [train set]  
3step: [validation] | [train set]  | [train set]  

In [5]:
# default is 5-fold cross validation 
cross = cross_validate(dt, train_input, train_target)

# Training time
train_fit_time = cross['fit_time']

# Evaluation time
train_score_time = cross['score_time']

# Score
train_score_list = cross['test_score']

print("Training time   : ", train_fit_time)
print("Evaluation time : ", train_score_time)
print("Score list (5)  ; ", train_score_list, end="\n\n")


# Best socre (mean)
print("Train Score:", np.mean(train_score_list))

Training time   :  [0.00728703 0.00786662 0.00744176 0.00728297 0.00697494]
Evaluation time :  [0.00076914 0.00069594 0.00070763 0.00065637 0.00066447]
Score list (5)  ;  [0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]

Train Score: 0.855300214703487


#### <b> StratifiedKFold </b>
split the data set

In [6]:
# StratifiedKFold
cross = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

# Training time
train_fit_time = cross['fit_time']

# Evaluation time
train_score_time = cross['score_time']

# Score
train_score_list = cross['test_score']

print("Training time   : ", train_fit_time)
print("Evaluation time : ", train_score_time)
print("Score list (5)  ; ", train_score_list, end="\n\n")

Training time   :  [0.00855231 0.00705862 0.00747895 0.00734735 0.00702167]
Evaluation time :  [0.00076056 0.00065684 0.0006423  0.00070667 0.00067616]
Score list (5)  ;  [0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]



#### <b> N-fold cross validation </b>
ex) 10-fold

In [7]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cross = cross_validate(dt, train_input, train_target, cv=splitter)

train_scores = cross['test_score']

scroe = np.mean(train_scores)

print(scroe)

0.8574181117533719


#### <b> Hyper Parameter </b>
Human-defined parameters  
'coefficient' and 'intercebpt' are not hyper parameter.   

<b> Parameter Tuning Process: </b>  
user defalut parameter value  
modift parameter with score or cros validation (1\~2) or (5\~6) iterate  
If parameter is not one, we have to fallow the below.  
such parameter value is influence each others. So If we want to modify the parameter,  We have to modify the all parameter 

In [11]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
dt = DecisionTreeClassifier(random_state=42) # decision tree 

# Declareation (defalut is 5-fold)
gs = GridSearchCV(dt, params, n_jobs=-1)

# cross validation training
gs.fit(train_input, train_target)

# best hyper parameter with Grid Search
dt = gs.best_estimator_

#### <b> Grid Seearrrch </b>
hyper parameter search + cross_validate()

####  <b> Score </b>
score = 